<a href="https://colab.research.google.com/github/PRISM-SING-HEALTH/2025-NCP-Projects/blob/main/NCP_Variant_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [241]:
import pandas as pd
import numpy as np
import yaml

In [242]:
def load_config(config_file):
  """
  Loads configuration from YAML file.

  Args:
    config_file (str): Path to the YAML configuration file.

  Returns:
    dict: Configuration settings.
  """

  with open(config_file, 'r') as file:
    config = yaml.safe_load(file)
  return config

# Load configuration
config = load_config('/content/drive/MyDrive/Variant_DB/Mock_Local_Drive/config.yaml')

# Initialise file paths for excel files
base_path = config['base_path']
local_path = config['local_path']
files = config['files']

In [243]:
# Define standard columns
standard_columns = [
    'MRN',
    'Patient Name',
    'Phenotype',
    'Solved Status',
    'Gene',
    'Transcript',
    'Variant',
    'HGVSg',
    'HGVSc',
    'HGVSp'
]

In [244]:
# Importing data into dataframes
print('Importing data...')
try:
  # Lab Cases
  lab_cases_df = pd.read_excel(
      f"{base_path}{files['lab_cases']}",
      sheet_name = 'Sheet1',
      header = 0,
      usecols = 'F, AF:AI, AL'
  )
  print('Passed.')

  # ATM Summary (fix)
  atm_summary_df = pd.read_excel(
      f"{base_path}{files['atm_summary']}",
      sheet_name='SUMMARY',
      header = None,
      usecols = 'A:B'
  )

  # Additional filtering due to unconventional formatting
  print('Performing additional filtering...')
  atm_summary_df_filtered = atm_summary_df.iloc[9:16]  # Ensure rows are correctly indexed
  atm_summary_df_filtered.reset_index(drop=True, inplace=True)  # Reset index for clean output

  # Convert to a structured DataFrame
  print('Transforming into structured DataFrame...')
  atm_structured_df = atm_summary_df_filtered.set_index(0).T  # Set column 0 as the header, transpose
  atm_structured_df.reset_index(drop=True, inplace=True)  # Reset index for clean final DataFrame

  # Invitae Summary
  invitae_summary_df = pd.read_excel(
      f"{base_path}{files['invitae_summary']}",
      sheet_name = 'Invitae list header',
      header = 0,
      usecols = 'E:F, M:T'
  )
  print('Passed.')

  # Clinical Summary
  clinical_summary_df = pd.read_excel(
      f"{base_path}{files['clinical_summary']}",
      sheet_name = '11 Dec',
      header = 0,
      usecols = 'D:E, K')
  print('Passed.')

  # Research Summary
  research_summary_df = pd.read_excel(
      f"{base_path}{files['research_summary']}",
      sheet_name = 'Overall List',
      header = 2,
      usecols = 'B, W:X, AF, AS:AT, AV:AW')
  print('Passed.')

except FileNotFoundError as e:
  print(f"File not found: {e}")
except ValueError as e:
  print(f"Error reading sheet or invalid data format: {e}")

print('Data import complete.')

Importing data...
Passed.
Performing additional filtering...
Transforming into structured DataFrame...
Passed.
Passed.
Passed.
Data import complete.


In [245]:
# Print dataframes
print('Printing dataframes...')

print('\nLab Cases')
print(lab_cases_df)

print('\nATM Summary')
print(atm_structured_df)

print('\nInvitae Summary')
print(invitae_summary_df)

print('\nClinical Summary')
print(clinical_summary_df)

print('\nResearch Summary')
print(research_summary_df)

Printing dataframes...

Lab Cases
                                           Phenotype Variant_1_gene  \
0  Developmental / Behavioural - Global developme...          PRRT2   
1  Developmental / Behavioural - Global developme...          PRRT2   
2  Developmental / Behavioural - Global developme...         STING1   
3  Developmental / Behavioural - Global developme...         DYRK1A   
4  Developmental / Behavioural - Delayed fine mot...          NTRK1   
5  Developmental / Behavioural - Global developme...         DYRK1A   
6  Developmental / Behavioural - Global developme...         DYRK1A   

               Variant_1_HGVSg          Variant_1_HGVSc  \
0          chr16:g.29813703dup     NM_145239.3:c.649dup   
1          chr16:g.29813703dup     NM_145239.3:c.649dup   
2  NC_000005.10:g.139480847C>T     NM_198282.4:c.463G>A   
3          chr21:g.37480775G>A  NM_001347721.2:c.438G>A   
4          chr1:g.156879282T>G    NM_002529.4:c.1966T>G   
5          chr21:g.37480775G>A  NM_00134772

In [246]:
# Standardise columns across dataframes
print('Standardising columns across dataframes...')

# Lab Cases
lab_cases_cols = {
    'Number variants detected'    :'Var Count',
    'Variant_1_gene'              :'Gene',
    'Variant_1_HGVSg'             :'HGVSg',
    'Variant_1_HGVSc'             :'HGVSc',
    'Variant_1_HGVSp'             :'HGVSp',
    'Variant_1_zygosity'          :'Zygosity',
    'Variant_1_inheritance'       :'Inheritance',
    'Variant_1_Validation_Status' :'Solved Status'
}

lab_cases_df.rename(columns = lab_cases_cols, inplace = True)
print('Passed.')

# ATM Summary
atm_structured_df_cols = {
    'HGVS_Genomic_GRCh38/hg38'            :'HSVSg',
    'HGVS_MANE Select_Transcript_RefSeq'  :'Transcript',
    'HGVS_MANE Select_cDNA'               :'HGVSc',
    'HGVS_MANE Select_protein'            :'HGVSp',
    'HUGO gene symbol'                    :'Gene',
}

atm_structured_df.rename(columns = atm_structured_df_cols, inplace = True)
print('Passed.')

# Invitae Summary
invitae_summary_df_cols = {
    'Patient ID (MRN)'  :'MRN',
    'Patient Name'      :'Patient Name',
    'Result'            :'Solved Status',
    'Gene'              :'Gene',
    'Transcript'        :'Transcript',
    'Variant'           :'Variant',
    'HGVSc'             :'HGVSc',
    'Protein Change'    :'HGVSp',
}

invitae_summary_df.rename(columns = invitae_summary_df_cols, inplace = True)
print('Passed.')

# Clinical Summary
clinical_summary_df_cols = {
    'Identification No.'        :'MRN',
    'Medical Prob description'  :'Phenotype',
    'Patient Name'              :'Patient Name'
}

clinical_summary_df.rename(columns = clinical_summary_df_cols, inplace = True)
print('Passed.')

# Research Summary
research_summary_df_cols = {
    'IC No (MRN)'         :'MRN',
    'Name'                :'Patient Name',
    'Candidate gene (1)'  :'Gene',
    'Transcript (1)'      :'Transcript',
    'cDNA (1)'            :'HGVSc',
    'Protein (1)'         :'HGVSp',
    'AUTO STATUS'         :'Solved Status'
}

research_summary_df.rename(columns = research_summary_df_cols, inplace = True)
print('Passed.')

Standardising columns across dataframes...
Passed.
Passed.
Passed.
Passed.
Passed.


In [247]:
# Print dataframes after renaming
print('Printing dataframes...')

print('\nLab Cases')
print(lab_cases_df)

print('\nATM Summary')
print(atm_structured_df)

print('\nInvitae Summary')
print(invitae_summary_df)

print('\nClinical Summary')
print(clinical_summary_df)

print('\nResearch Summary')
print(research_summary_df)

Printing dataframes...

Lab Cases
                                           Phenotype    Gene  \
0  Developmental / Behavioural - Global developme...   PRRT2   
1  Developmental / Behavioural - Global developme...   PRRT2   
2  Developmental / Behavioural - Global developme...  STING1   
3  Developmental / Behavioural - Global developme...  DYRK1A   
4  Developmental / Behavioural - Delayed fine mot...   NTRK1   
5  Developmental / Behavioural - Global developme...  DYRK1A   
6  Developmental / Behavioural - Global developme...  DYRK1A   

                         HGVSg                    HGVSc  \
0          chr16:g.29813703dup     NM_145239.3:c.649dup   
1          chr16:g.29813703dup     NM_145239.3:c.649dup   
2  NC_000005.10:g.139480847C>T     NM_198282.4:c.463G>A   
3          chr21:g.37480775G>A  NM_001347721.2:c.438G>A   
4          chr1:g.156879282T>G    NM_002529.4:c.1966T>G   
5          chr21:g.37480775G>A  NM_001347721.2:c.438G>A   
6          chr21:g.37480775G>A  NM_00134

In [248]:
def standardise_dataframe(df, columns):
  """
  Converts unstandardised dataframe to standardised dataframe.

  Args:
    df (pd.DataFrame): Input dataframe.
    columns (list): List of standardised column names.

  Returns:
    standardised_df (pd.DataFrame): Standardized dataframe.
  """
  standardised_df = df.reindex(columns = columns, fill_value = np.nan)
  return standardised_df

print('Standardising all dataframes...')

lab_cases_df_standard = standardise_dataframe(lab_cases_df, standard_columns)
atm_structured_df_standard = standardise_dataframe(atm_structured_df, standard_columns)
invitae_summary_df_standard = standardise_dataframe(invitae_summary_df, standard_columns)
clinical_summary_df_standard = standardise_dataframe(clinical_summary_df, standard_columns)
research_summary_df_standard = standardise_dataframe(research_summary_df, standard_columns)

print('Standardisation complete.')

Standardising all dataframes...
Standardisation complete.


In [249]:
# Print dataframes to verify
print('Individual dataframes:')
print('\nLab Cases:')
print(lab_cases_df_standard.head())

print('\nATM:')
print(atm_structured_df_standard.head())

print('\nInvitae:')
print(invitae_summary_df_standard.head())

print('\nClinical:')
print(clinical_summary_df_standard.head())

print('\nResearch:')
print(research_summary_df_standard.head())

Individual dataframes:

Lab Cases:
   MRN  Patient Name                                          Phenotype  \
0  NaN           NaN  Developmental / Behavioural - Global developme...   
1  NaN           NaN  Developmental / Behavioural - Global developme...   
2  NaN           NaN  Developmental / Behavioural - Global developme...   
3  NaN           NaN  Developmental / Behavioural - Global developme...   
4  NaN           NaN  Developmental / Behavioural - Delayed fine mot...   

       Solved Status    Gene  Transcript  Variant  \
0  Confirmed present   PRRT2         NaN      NaN   
1  Confirmed present   PRRT2         NaN      NaN   
2  Confirmed present  STING1         NaN      NaN   
3  Confirmed present  DYRK1A         NaN      NaN   
4  Confirmed present   NTRK1         NaN      NaN   

                         HGVSg                    HGVSc  \
0          chr16:g.29813703dup     NM_145239.3:c.649dup   
1          chr16:g.29813703dup     NM_145239.3:c.649dup   
2  NC_000005.10:g.

In [250]:
# Combining dataframes into a single dataframe
print('\nCombining dataframes...')
combined_df = pd.concat([
    lab_cases_df_standard,
    atm_structured_df_standard,
    invitae_summary_df_standard,
    clinical_summary_df_standard,
    research_summary_df_standard
], ignore_index = True)

print('All dataframes combined successfully.')

print('Combined dataframe:')
print(combined_df)


Combining dataframes...
All dataframes combined successfully.
Combined dataframe:
        MRN Patient Name                                          Phenotype  \
0       NaN          NaN  Developmental / Behavioural - Global developme...   
1       NaN          NaN  Developmental / Behavioural - Global developme...   
2       NaN          NaN  Developmental / Behavioural - Global developme...   
3       NaN          NaN  Developmental / Behavioural - Global developme...   
4       NaN          NaN  Developmental / Behavioural - Delayed fine mot...   
5       NaN          NaN  Developmental / Behavioural - Global developme...   
6       NaN          NaN  Developmental / Behavioural - Global developme...   
7       NaN          NaN                                                NaN   
8   43234.0    John Pork                                                NaN   
9       NaN          NaN                                                NaN   
10    420.0      Jon Doe  Shwachman-Diamond Synd

In [ ]:
def query(df):
  """
  Queries dataframe based on user input.

  Args:
    df (pd.DataFrame): Input dataframe.

  Returns:
    query_result (pd.DataFrame): Query result.
  """
  print('\n### Query Dataframe ###')
  print(f"Available categories: {', '.join(df.columns)}\n")

  try:
    # User input category
    category = input('Enter category: ').strip()
    category_lower = category.lower() # convert to lower case for non-case sens.

    # Check if category exists
    categories_lower = [col.lower() for col in df.columns]
    if category_lower not in categories_lower:
      raise ValueError(f"Invalid category '{category}'. Please choose from available categories.")

    # Find actual column name (case sens match)
    category_actual = df.columns[categories_lower.index(category_lower)]

    # User input item (case insensitive and substring search)
    item = input(f"Enter item in category '{category_actual}': ").strip()

    # Query dataframe with case insensitive comparison
    query_result = df[df[category_actual].astype(str).str.contains(item, case = False, na = False)]

    if query_result.empty:
      print(f"No results found for item containing '{item}' in category '{category_actual}'.")
    else:
      print(f"\nQuery Results for item ({len(query_result)} records(s) found):")
      return query_result

  except ValueError as e:
    print(f"Error: {e}")
    return None

query_result = query(combined_df)


### Query Dataframe ###
Available categories: MRN, Patient Name, Phenotype, Solved Status, Gene, Transcript, Variant, HGVSg, HGVSc, HGVSp



In [ ]:
def export_to_excel(df, output_path):
    """
    Exports the given DataFrame to an Excel file.

    Args:
        df (pd.DataFrame): The DataFrame to export.
        output_path (str): Path where the Excel file will be saved.

    Returns:
        None
    """
    try:
        df_filled = df.fillna('NA')
        df_filled.to_excel(output_path, index=False)
        print(f"Data successfully exported to {output_path}")
    except Exception as e:
        print(f"An error occurred while exporting to Excel: {e}")

output_file = f"{local_path}Results/combined_data.xlsx"
export_to_excel(query_result, output_file)